In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\0073797c-a755-4972-b76b-8ef2b31d44ab_...,140,339,210,260
1,./images\03273806-bb1e-48da-8c8b-a0133a90197a_...,185,374,290,339
2,./images\0369b20e-b432-4409-90f9-2420877aa386_...,335,453,313,347
3,./images\07064c2c-2aa3-4419-91a4-92916de8e54c_...,205,335,290,353
4,./images\0850c175-0b8d-47f2-801c-e29f1dbdb367_...,374,596,275,382


In [3]:
import xml.etree.ElementTree as xet

In [8]:
filename = df["filepath"][0]
filename

'./images\\0073797c-a755-4972-b76b-8ef2b31d44ab___new_IMG_20160315_071740.jpg.xml'

In [10]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [11]:
getFilename(filename)

'./images\\0073797c-a755-4972-b76b-8ef2b31d44ab___new_IMG_20160315_071740.jpg.jpeg'

In [14]:
image_path = list(df['filepath'].apply(getFilename))

In [15]:
image_path

['./images\\0073797c-a755-4972-b76b-8ef2b31d44ab___new_IMG_20160315_071740.jpg.jpeg',
 './images\\03273806-bb1e-48da-8c8b-a0133a90197a___2014-Skoda-Yeti-Test-Drive.jpg.jpeg',
 './images\\0369b20e-b432-4409-90f9-2420877aa386___8151536c79159a1557421da5f27f9f0e.jpg.jpeg',
 './images\\07064c2c-2aa3-4419-91a4-92916de8e54c___mahindra-scorpio-old-car-500x500.jpg.jpeg',
 './images\\0850c175-0b8d-47f2-801c-e29f1dbdb367___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_571.jpg',
 './images\\092c497e-623d-499d-822a-4b01c657389b___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_nissan-terrano-rear-left-rim.jpg',
 './images\\11186c4c-5f17-49b1-af37-a138a1e53794___12837944763_e7c6c6381a_z.jpg.jpeg',
 './images\\11832b13-d514-4f1d-967c-d00e76d21e9b___Yellow-Number-Plate-With-Black-Lettering.jpg.jpeg',
 './images\\125b5cda-9691-47aa-8e57-f6d8a8537d83___fancy_number_plates_in_india.jpg.jpeg',
 './images\\1438b735-2d21-4879-8ed4-e5fa086e70c8___Maruti-Suzuki-Swift-Dzire-1.jpg.jpeg',
 './images\\165eaa92-8e90-44e6-b132-8af462b

#### verify image data

In [17]:
file_path = image_path[0]
file_path

'./images\\0073797c-a755-4972-b76b-8ef2b31d44ab___new_IMG_20160315_071740.jpg.jpeg'

In [19]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
# 140	339	210	260
cv2.rectangle(img,(140,210),(339,260),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Data Preprocessing

In [21]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [24]:
labels = df.iloc[:,1:].values

In [28]:
len(image_path)

1664

In [32]:
data= []
output=[]

for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    if img_arr is None:
        continue
    h, w, d = img_arr.shape

   # preprocessing
    load_image = cv2.resize(img_arr, (224, 224))
    load_image_arr = np.array(load_image)
    norm_load_image_arr = load_image_arr / 255.0  # normalization

    # normalization to labels
    xmin, xmax, ymin, ymax = labels[ind]
    nxmin, nxmax = xmin / w, xmax / w
    nymin, nymax = ymin / h, ymax / h
    labels_norm = (nxmin, nxmax, nymin, nymax)  # normalization output

    # append
    data.append(norm_load_image_arr)
    output.append(labels_norm)

In [34]:
# labels_norm

In [35]:
# load_image_arr

In [40]:
X = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)

In [41]:
X.shape, y.shape

((1579, 224, 224, 3), (1579, 4))

In [42]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.8,random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((315, 224, 224, 3), (1264, 224, 224, 3), (315, 4), (1264, 4))

### Deep Learning Model

In [ ]:
#transfer learning

In [43]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [44]:
inception_resnet = InceptionResNetV2(weights="imagenet", include_top=False,
                                    input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable = False

headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation="relu")(headmodel)
headmodel = Dense(250, activation="relu")(headmodel)
headmodel = Dense(4, activation='sigmoid')(headmodel)

model = Model(inputs  =inception_resnet.input,outputs=headmodel)

219070464/219055592 [==============================] - 26s 0us/step


In [45]:
#compile model

model.compile(loss='mse',optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 batch_normalization_5 (BatchNo  (None, 25, 25, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 25, 25, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_10 (BatchN  (None, 25, 25, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 25, 25, 64)  192         ['conv2d_11[0][0]']              
 ormalizat

 activation_21 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 25, 25, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 25, 25, 48)   13824       ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_19 (BatchN  (None, 25, 25, 32)  96          ['conv2d_19[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_22 (BatchN  (None, 25, 25, 48)  144         ['conv2d_22[0][0]']              
 ormalizat

                                                                                                  
 activation_24 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 25, 25, 64)   0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 25, 25, 128)  0           ['activation_24[0][0]',          
                                                                  'activation_26[0][0]',          
                                                                  'activation_29[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_40 (BatchN  (None, 25, 25, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_37 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_37[0][0]'] 
                                                                                                  
 activation_40 (Activation)     (None, 25, 25, 48)   0           ['batch_normalization_40[0][0]'] 
                                                                                                  
 conv2d_36 (Conv2D)             (None, 25, 25, 32)   10240       ['block35_4_ac[0][0]']           
          

...

                                                                                                  
 activation_169 (Activation)    (None, 5, 5, 224)    0           ['batch_normalization_169[0][0]']
                                                                                                  
 conv2d_167 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_170 (Conv2D)            (None, 5, 5, 256)    172032      ['activation_169[0][0]']         
                                                                                                  
 batch_normalization_167 (Batch  (None, 5, 5, 192)   576         ['conv2d_167[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_nor

 batch_normalization_175 (Batch  (None, 5, 5, 192)   576         ['conv2d_175[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_178 (Batch  (None, 5, 5, 256)   768         ['conv2d_178[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_175 (Activation)    (None, 5, 5, 192)    0           ['batch_normalization_175[0][0]']
                                                                                                  
 activation_178 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_178[0][0]']
                                                                                                  
 block8_4_

                                                                                                  
 activation_186 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_186[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 5, 5, 448)    0           ['activation_183[0][0]',         
                                                                  'activation_186[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 5, 5, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 block8_6 (Lambda)              (None, 5, 5, 2080)   0           ['block8_5_ac[0][0]',            
                                                                  'block8_6_conv[0][0]']          
          

                                                                                                  
 block8_8 (Lambda)              (None, 5, 5, 2080)   0           ['block8_7_ac[0][0]',            
                                                                  'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 5, 5, 2080)   0           ['block8_8[0][0]']               
                                                                                                  
 conv2d_196 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_normalization_196 (Batch  (None, 5, 5, 192)   576         ['conv2d_196[0][0]']             
 Normalization)                                                                                   
          

 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 5, 5, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
 flatten (Flatten)              (None, 38400)        0           ['conv_7b_ac[0][0]']             
                                                                                                  
 dense (Dense)                  (None, 500)          19200500    ['flatten[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 250)          125250      ['dense[0][0]']                  
                                                                                                  
 dense_2 (

In [46]:
#model training

from tensorflow.keras.callbacks import TensorBoard

In [47]:
tfb = TensorBoard('object_detection')

In [48]:
history = model.fit(x=x_train, y=y_train,batch_size=10, epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
32/32 [==============================] - 191s 6s/step - loss: 0.0956 - val_loss: 0.0854
Epoch 2/100
32/32 [==============================] - 192s 6s/step - loss: 0.0864 - val_loss: 0.0799
Epoch 3/100
32/32 [==============================] - 222s 7s/step - loss: 0.0822 - val_loss: 0.0756
Epoch 4/100
32/32 [==============================] - 251s 8s/step - loss: 0.0756 - val_loss: 0.0760
Epoch 5/100
32/32 [==============================] - 258s 8s/step - loss: 0.0773 - val_loss: 0.0729
Epoch 6/100
32/32 [==============================] - 282s 9s/step - loss: 0.0730 - val_loss: 0.0718
Epoch 7/100
32/32 [==============================] - 239s 8s/step - loss: 0.0724 - val_loss: 0.0727
Epoch 8/100
32/32 [==============================] - 258s 8s/step - loss: 0.0709 - val_loss: 0.0701
Epoch 9/100
32/32 [==============================] - 237s 8s/step - loss: 0.0569 - val_loss: 0.0541
Epoch 10/100
32/32 [==============================] - 247s 8s/step - loss: 0.0462 - val_loss: 0.0466

In [ ]:
# history = model.fit(x=x_train, y=y_train,batch_size=10, epochs=100,
#                     validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

### Save the Model

In [49]:
model.save('./models/object_detection.h5')

### Visualise on Tensorboard

In [62]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [63]:
logdir = "logs/object_detection"

In [64]:
%tensorboard --logdir {}

Reusing TensorBoard on port 6006 (pid 24036), started 0:01:05 ago. (Use '!kill 24036' to kill it.)

- cmd> New Folder
- tensorboard --logdir="object _detection"